In [ ]:
import os
import tempfile
import sbol3
import paml
import tyto
import uml
import json
import rdflib as rdfl
from IPython.display import Markdown

from paml.execution_engine import ExecutionEngine
from paml_check.paml_check import check_doc

import logging
logging.basicConfig(level=logging.DEBUG)
l = logging.getLogger()

%load_ext autoreload
%autoreload 2

In [ ]:
# Setup PAML protocol for execution
out_dir = "."
doc = sbol3.Document()
sbol3.set_namespace('https://bbn.com/scratch/')
protocol_file = os.path.join(os.getcwd(), "../test/testfiles", "igem_ludox_test.nt")
doc.read(protocol_file, 'nt')
protocol = doc.find("https://bbn.com/scratch/iGEM_LUDOX_OD_calibration_2018")
agent = sbol3.Agent("test_agent")


# Execute the protocol
ee = ExecutionEngine()
parameter_values = [
    paml.ParameterValue(parameter=protocol.get_input("wavelength"), 
                        value=uml.LiteralIdentified(value=sbol3.Measure(100, tyto.OM.nanometer)))
]
execution = ee.execute(protocol, agent, id="test_execution", parameter_values=parameter_values)

In [ ]:
data = execution.get_data()
#data
data['https://bbn.com/scratch/execute_6/ParameterValue3'].from_samples.lookup().source.lookup().contents.elements[1].property_value

In [ ]:
# Programmatically update the SampleData object and save execution

key = 'https://bbn.com/scratch/execute_6/ParameterValue3'

df = data[key].to_dataframe()
df["values"] = df["values"].apply(lambda x: 8)
data[key].from_dataframe(df, "df.csv")

v = doc.validate()
assert len(v) == 0, "".join(f'\n {e}' for e in v)
temp_name = os.path.join(".", 'igem_ludox_test_exec_data.nt')#os.path.join(tempfile.gettempdir(), 'igem_ludox_test_exec_data.nt')
doc.write(temp_name, sbol3.SORTED_NTRIPLES)

In [ ]:
# Use pandasgui to graphically update SampleData object

import pandas as pd
from pandasgui import show

dfs = {k:v.to_dataframe() for k, v in data.items()}
gui = show(**dfs)

In [ ]:
dfs = gui.get_dataframes()
data = {k: v.from_dataframe(dfs[k], "df.csv") for k, v in data.items()}
data

In [ ]:
data

In [ ]:
execution.to_dot()